In [1]:
import sys    
print('Running on Python version: {}'.format(sys.version))

try:
    import lazyarray
except:
    !pip install lazyarray
    !pip install hdbscan 
    !pip install scipy --upgrade
    !pip install numpy numba --upgrade 
    !pip install lmfit 

Running on Python version: 3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


In [2]:
import os
import pickle

try:
    izhi_opt = pickle.load(open('protected/multi_objective_izhi.p','rb')) 
except:
    
    os.system('wget wget -O ncp_cell_layer_5_6.p https://osf.io/6yba2/download')
    os.system('wget -O multi_objective_izhi https://osf.io/3vp8d/download')
    izhi_opt = pickle.load(open('multi_objective_izhi.p','rb')) 





/opt/conda/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/opt/conda/lib/python3.5/site-packages/pyNN/neuron/__init__.py:14: UserWarning: mpi4py not available
  warnings.warn("mpi4py not available")


# Get Data
If content is not available locally, wget the files from the Open Science Frame Work.

In [5]:
from neuronunit.tests.druckmann2013 import *


In [6]:

dtc0 = list(izhi_opt.values())[0]['pf'][0].dtc
print(list(izhi_opt.values())[0]['pf'][0].dtc.scores)
print(list(izhi_opt.values())[0]['pf'][-1].dtc.scores)
print(list(izhi_opt.values())[0]['pf'][2].dtc.scores)
print(list(izhi_opt.values())[0]['pf'][1].dtc.scores)
list(izhi_opt.values())[0]['pf'][0].dtc.attrs

print(dtc0.rheobase)
ir_currents = {}
ir_currents = dtc0.rheobase
standard = 1.5*ir_currents
standard*=1.5
strong = 3*ir_currents

{'InjectedCurrentAPThresholdTest': 0.9999383647108253, 'RestingPotentialTest': 0.0010068761003825255, 'InputResistanceTest': 0.5162167574928647, 'InjectedCurrentAPAmplitudeTest': 1.0, 'RheobaseTest': 0.43475020334261383, 'TimeConstantTest': 0.19463622334314878, 'InjectedCurrentAPWidthTest': 0.004511111383472155}
{'InjectedCurrentAPThresholdTest': 0.9957958143881491, 'RestingPotentialTest': 0.9743806289694379, 'InputResistanceTest': 0.7583867186758638, 'InjectedCurrentAPAmplitudeTest': 1.0, 'RheobaseTest': 0.27439091965608964, 'TimeConstantTest': 0.9985452095287137, 'InjectedCurrentAPWidthTest': 0.8045887465032604}
{'InjectedCurrentAPThresholdTest': 0.9999999998992437, 'RestingPotentialTest': 0.046204212239741804, 'InputResistanceTest': 0.5587581971196718, 'InjectedCurrentAPAmplitudeTest': 1.0, 'RheobaseTest': 0.34361838339745054, 'TimeConstantTest': 0.07139840065746239, 'InjectedCurrentAPWidthTest': 0.16128647828914078}
{'InjectedCurrentAPThresholdTest': 0.9999999999994738, 'RestingPot

In [7]:

tests = [AP12AmplitudeDropTest(standard), 
    AP1SSAmplitudeChangeTest(standard), 
    AP1AmplitudeTest(standard), 
    AP1WidthHalfHeightTest(standard), 
    AP1WidthPeakToTroughTest(standard), 
    AP1RateOfChangePeakToTroughTest(standard), 
    AP1AHPDepthTest(standard), 
    AP2AmplitudeTest(standard), 
    AP2WidthHalfHeightTest(standard), 
    AP2WidthPeakToTroughTest(standard), 
    AP2RateOfChangePeakToTroughTest(standard), 
    AP2AHPDepthTest(standard), 
    AP12AmplitudeChangePercentTest(standard), 
    AP12HalfWidthChangePercentTest(standard), 
    AP12RateOfChangePeakToTroughPercentChangeTest(standard), 
    AP12AHPDepthPercentChangeTest(standard), 
    AP1DelayMeanTest(standard), 
    AP1DelaySDTest(standard), 
    AP2DelayMeanTest(standard), 
    AP2DelaySDTest(standard), 
    Burst1ISIMeanTest(standard), 
    Burst1ISISDTest(standard), 
    InitialAccommodationMeanTest(standard), 
    SSAccommodationMeanTest(standard), 
    AccommodationRateToSSTest(standard), 
    AccommodationAtSSMeanTest(standard), 
    AccommodationRateMeanAtSSTest(standard), 
    ISICVTest(standard), 
    ISIMedianTest(standard), 
    ISIBurstMeanChangeTest(standard), 
    SpikeRateStrongStimTest(strong), 
    AP1DelayMeanStrongStimTest(strong), 
    AP1DelaySDStrongStimTest(strong), 
    AP2DelayMeanStrongStimTest(strong), 
    AP2DelaySDStrongStimTest(strong), 
    Burst1ISISDStrongStimTest(strong),
    Burst1ISIMeanStrongStimTest(strong)]

AHP_list = [AP1AHPDepthTest(standard), 
    AP2AHPDepthTest(standard), 
    AP12AHPDepthPercentChangeTest(standard) ] 


In [ ]:
print(list(izhi_opt.values())[0]['pf'][1].dtc.scores)
list(izhi_opt.values())[0]['pf'][0].dtc.attrs

print(dtc0.rheobase)
ir_currents = {}
ir_currents = dtc0.rheobase
standard = 1.5*ir_currents
standard*=1.5
strong = 3*ir_currents
print(strong)
np.isin = np.in1d

from neuronunit.optimisation.optimisation_management import mint_generic_model

import quantities as pq
current_amplitude = {'mean': ir_currents, 'n': 1, 'std': 0.0 * pq.pA}
test = AP12AmplitudeChangePercentTest(current_amplitude*100)


model = mint_generic_model(dtc0.backend)
model.set_attrs(dtc0.attrs)
model = ReducedModel(LEMS_MODEL_PATH, name= str('vanilla'), backend=('RAW'),dtc)
model
    
for i, test in enumerate(tests):
    mean = test.generate_prediction(model)['mean']
    
    


In [ ]:

from pandas import DataFrame
import numpy as np
import pandas
from matplotlib import pyplot as plt

import string 
from scipy.stats import pearsonr
import statsmodels.formula.api as smf
import hdbscan
from sklearn.cluster import KMeans
from scipy.spatial.distance import euclidean

pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', 20)
from manager import ModelManager
mgr = ModelManager()
mgr.server.connect()

In [ ]:
protocol_cost = {
    'Steady State':       1, # 1s
    'Standard':         5*2, # 5 repetitions of 1s SS 1s stim
    'Strong':           5*2, # 5 repetitions of 1s SS 1s strong stim
    'Input Resistance': 2*2, # 2 levels of 1s SS 1s stim
}

prop_protocol = {
    'AP1Amplitude': 'Standard',
    'AP2Amplitude': 'Standard',
    'AP12AmplitudeDrop': 'Standard',
    'AP12AmplitudeChangePercent': 'Standard',
    'AP1SSAmplitudeChange': 'Standard'  ,
    
    'AP1WidthHalfHeight': 'Standard',
    'AP2WidthHalfHeight': 'Standard',
    'AP12HalfWidthChangePercent': 'Standard',
    
    'AP1WidthPeakToTrough': 'Standard',
    'AP2WidthPeakToTrough': 'Standard',
    
    'AP1RateOfChangePeakToTrough': 'Standard',
    'AP2RateOfChangePeakToTrough': 'Standard', 
    'AP12RateOfChangePeakToTroughPercentChange': 'Standard',
    
    'AP1AHPDepth': 'Standard',
    'AP2AHPDepth': 'Standard',
    'AP12AHPDepthPercentChange': 'Standard',
    
    'AP1DelayMean': 'Standard',
    'AP2DelayMean': 'Standard',
    
    'AP1DelaySD': 'Standard',
    'AP2DelaySD': 'Standard',
    
    'AP1DelayMeanStrongStim': 'Strong',
    'AP2DelayMeanStrongStim': 'Strong',
    
    'AP1DelaySDStrongStim': 'Strong',
    'AP2DelaySDStrongStim': 'Strong',
    
    'Burst1ISIMean': 'Standard',
    'Burst1ISIMeanStrongStim': 'Strong',
    
    'Burst1ISISD': 'Standard',
    'Burst1ISISDStrongStim': 'Strong',
    
    'InitialAccommodationMean': 'Standard',
    'SSAccommodationMean': 'Standard',
    'AccommodationRateToSS': 'Standard',
    'AccommodationAtSSMean': 'Standard',
    'AccommodationRateMeanAtSS': 'Standard',
    
    
    'ISIMedian': 'Standard',
    'ISICV': 'Standard',
    'ISIBurstMeanChange': 'Standard',
    
    'SpikeRateStrongStim': 'Strong',
    
    'InputResistance': 'Input Resistance',
    
    'SteadyStateAPs': 'Steady State',
}

prop_names = [
    'AP1Amplitude',
    'AP2Amplitude',
    'AP12AmplitudeDrop',
    'AP12AmplitudeChangePercent',
    'AP1SSAmplitudeChange',  
    
    'AP1WidthHalfHeight',
    'AP2WidthHalfHeight',
    'AP12HalfWidthChangePercent',
    
    'AP1WidthPeakToTrough',
    'AP2WidthPeakToTrough',
    
    'AP1RateOfChangePeakToTrough',
    'AP2RateOfChangePeakToTrough',    
    'AP12RateOfChangePeakToTroughPercentChange',
    
    'AP1AHPDepth',
    'AP2AHPDepth',
    'AP12AHPDepthPercentChange',
    
    'AP1DelayMean',
    'AP2DelayMean',
    
    'AP1DelaySD',
    'AP2DelaySD',
    
    'AP1DelayMeanStrongStim',
    'AP2DelayMeanStrongStim',
    
    'AP1DelaySDStrongStim',
    'AP2DelaySDStrongStim',
    
    'Burst1ISIMean',
    'Burst1ISIMeanStrongStim',
    
    'Burst1ISISD',
    'Burst1ISISDStrongStim',
    
    'InitialAccommodationMean',
    'SSAccommodationMean',
    'AccommodationRateToSS',
    'AccommodationAtSSMean',
    'AccommodationRateMeanAtSS',
    
    
    'ISIMedian',
    'ISICV',
    'ISIBurstMeanChange',
    
    'SpikeRateStrongStim',
    
    'InputResistance',
    
    'SteadyStateAPs',
    
    'FrequencyPassAbove',
    'FrequencyPassBelow',
    
    'RampFirstSpike',
]